# Zero Shot Classification

Estos modelos llamados **zero-shot** intentan clasificar texto o imagenes en clases para los cuales no han sido entrenados, es decir, sin la tecnica de finetuning que veremos en la siguiente unidad.

In [ ]:
!pip install transformers[torch] --quiet

In [ ]:
from transformers import pipeline

In [ ]:
# device=0 usara la GPU y device=-1 usara la CPU
classifier = pipeline(task = "zero-shot-image-classification", 
                      model = "openai/clip-vit-large-patch14",
                      device = 0)

In [ ]:
from PIL import Image
import requests
url = "https://images.unsplash.com/photo-1494790108377-be9c29b29330"
image = Image.open(requests.get(url, stream=True).raw)
#print(image.size)
image.thumbnail((600,300),Image.LANCZOS)
image

In [ ]:
predictions = classifier(image, candidate_labels=["anger", "contemp", "disgust", "fear", "happy","neutral","sad","surprise"])

In [ ]:
predictions

## Sin pipelines

In [ ]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
checkpoint = "openai/clip-vit-large-patch14"
model = AutoModelForZeroShotImageClassification.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

In [ ]:
candidate_labels = ["tree", "car", "bike", "cat"]
inputs = processor(images=image, text=candidate_labels, return_tensors="pt", padding=True)

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

In [ ]:
result = [
    {"score": score, "label": candidate_label}
    for score, candidate_label in sorted(zip(probs, candidate_labels), key=lambda x: -x[0])
]

## Ejercicios

### Ejercicio 1:

Clasificar las [imagenes de bananas maduras](https://github.com/amiune/freecodingtour/raw/main/cursos/espanol/deeplearning/data/bananas/) y frescas utilizando un modelo de zero-shot

In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 1:

import requests
from PIL import Image
from transformers import pipeline
classifier = pipeline(task = "zero-shot-image-classification", 
                      model = "openai/clip-vit-large-patch14",
                      device = -1)

# Lista de nombres de las imagenes
nombres_imagenes = [f'normal{i}.jpeg' for i in range(1,6)]
nombres_imagenes.extend([f'madura{i}.jpeg' for i in range(1,6)])
PATH = 'https://github.com/amiune/freecodingtour/raw/main/cursos/espanol/deeplearning/data/bananas/'
for nombre_imagen in nombres_imagenes:
    response = requests.get(PATH + nombre_imagen)
    with Image.open(BytesIO(response.content)) as img:
        predictions = classifier(image, candidate_labels=["banana","ripe banana"])
        print(nombre_imagen, predictions)

### Ejercicio 2:

Clasificar las siguientes peliculas por genero, terror o comedia, utilizando zero shot classification para texto. Probar diferentes modelos y comparar sus resultados.

Comedia: 
- Risa en el paraíso (1987)
- Risa y lágrimas (1954)
- La vida en risa (1996)
- El club de la risa (1999)
- Divertido, divertido (1969)
- Divertida aventura (1964)
- Divertida mente (2015)
- Diversión con Bob (1994)
- Locura de amor (1948)
- Locura en el oeste (1973)
- Locura de verano (1986)
- Locura en el espacio (1987)
- La risa es contagiosa (1969)
- Diversión asegurada (2003)
- Locura y dinero (2008)

Terror:
- Miedo (2007)
- Miedo en la noche (1972)
- Miedo a volar (1990)
- La matanza de Texas (1974)
- La matanza de Texas 2 (1986)
- La matanza de Texas: El origen (2007)
- Matanza en el colegio (1982)
- La masacre de los inocentes (1991)
- Masacre en la autopista (1977)
- Masacre en el hospital (1982)
- Masacre de zombies (1978)
- Miedo profundo (1998)
- Matanza en Fresno (2016)
- La masacre de Amityville (1979)


In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 2: Paso 1 {display-mode:"form"}

# Como el texto no esta en formato de tabla 
# uso regex para darle el formato que quiero
# Consejo: utilizar chatgpt o similar para obtener la regex correcta
import re
def get_titles(text):
    ret = []
    for title in text.split('\n'):
        clean = re.sub(r'^-\s+|\s+\([^)]*\)$', '', title)
        ret.append(clean)
    return ret

comedias = """- Risa en el paraíso (1987)
- Risa y lágrimas (1954)
- La vida en risa (1996)
- El club de la risa (1999)
- Divertido, divertido (1969)
- Divertida aventura (1964)
- Divertida mente (2015)
- Diversión con Bob (1994)
- Locura de amor (1948)
- Locura en el oeste (1973)
- Locura de verano (1986)
- Locura en el espacio (1987)
- La risa es contagiosa (1969)
- Diversión asegurada (2003)
- Locura y dinero (2008)"""

terror = """- Miedo (2007)
- Miedo en la noche (1972)
- Miedo a volar (1990)
- La matanza de Texas (1974)
- La matanza de Texas 2 (1986)
- La matanza de Texas: El origen (2007)
- Matanza en el colegio (1982)
- La masacre de los inocentes (1991)
- Masacre en la autopista (1977)
- Masacre en el hospital (1982)
- Masacre de zombies (1978)
- Miedo profundo (1998)
- Matanza en Fresno (2016)
- La masacre de Amityville (1979)"""


clean_titles_comedias = get_titles(comedias)
X = clean_titles_comedias
y = ['comedia'] * len(clean_titles_comedias)

clean_titles_terror = get_titles(terror)
X.extend(clean_titles_terror)
y.extend(['terror'] * len(clean_titles_terror))

import pandas as pd
df = pd.DataFrame({"Titulo": X,"Genero": y})
df

In [ ]:
#@title Solucion Ejercicio 2: Paso 2 {display-mode:"form"}

# Probamos un modelo zero shot con la primera pelicula
from transformers import pipeline
clasificador = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)

texto_a_clasificar = df.iloc[0,:]["Titulo"]
posibles_clases = ["comedia","terror"]
clasificador(texto_a_clasificar, posibles_clases)

In [ ]:
#@title Solucion Ejercicio 2: Paso 3 {display-mode:"form"}

# Probamos el modelo con todas las peliculas y calculamos el accuracy
N = len(df)
accuracy = 0
for i in range(N):
    result = clasificador(df.iloc[i,:]["Titulo"],posibles_clases)
    print(f"{i}: Real:{df.iloc[i,:]['Genero']} Predicho:{result['labels'][0]}")
    if df.iloc[i,:]['Genero'] == result['labels'][0]:
        accuracy += 1
print(f"Total accuracy:{accuracy/N}")

### Ejercicio 3:

1. Probar la solucion anterior con el siguiente modelo:https://huggingface.co/Recognai/bert-base-spanish-wwm-cased-xnli. Analizar el resultado y los problemas que tiene.

2. Probar el siguiente modelo: https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7 

3. Probar tambien la diferencia entre utilizar device=-1 y device=0 en la clase pipeline. Analizar el resultado de ambas ejecuciones.

In [ ]:
#@title Solucion Ejercicio 3: {display-mode:"form"}

# Aqui veremos que el modelo funciona mas rapido al estar en la
# GPU (device=0) que al estar en la CPU (device=-1)
# Tambien vemos que el modelo preentrenado en español
# funciona mejor aunque asocia locura con terror
# El segundo modelo predice todo correctamente

# Probamos un modelo zero shot con la primera pelicula
from transformers import pipeline
clasificador = pipeline("zero-shot-classification", model="Recognai/bert-base-spanish-wwm-cased-xnli", device=-1)
#clasificador = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", device=-1)

N = len(df)
accuracy = 0
for i in range(N):
    result = clasificador(df.iloc[i,:]["Titulo"],posibles_clases)
    print(f"{i}: Real:{df.iloc[i,:]['Genero']} Predicho:{result['labels'][0]}")
    if df.iloc[i,:]['Genero'] == result['labels'][0]:
        accuracy += 1
print(f"Total accuracy:{accuracy/N}")

### Ejercicio 4:

Puedes usar el siguiente codigo para entrenar tu propia red neuronal para clasificacion de imagenes?

https://github.com/nateraw/huggingpics

In [ ]:
# Escribir aqui la solucion



### Referencias
- https://huggingface.co/docs/transformers/tasks/zero_shot_image_classification

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)